# Prompt sensitivity

We use OpenAI's o3 model to generate 20  versions of the SCE-eliciting prompts for the house prices dataset. This notebook outlines how we generated the perturbed prompts. It outlines the prompts given to o3, but in practice, this was done through the web interface rather than the API. The perturbed prompts are added to the dataset.json file as `house_prices_followup_0`, `house_prices_followup_1`,...etc, and as `house_prices_minimal_followup_0`, `house_prices_minimal_followup_1`,...etc

In [12]:
import json
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk
from tqdm import tqdm
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
import numpy as np
from scipy import stats
from collections import Counter, defaultdict

import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")
from config import REPO_ROOT

In [13]:
# Load JSON into the `datasets` variable
with open(REPO_ROOT / 'src/models_datasets/datasets.json', 'r') as json_file:
    datasets = json.load(json_file)

#### Standard house prices prompts

In [14]:
house_prices = ("""You will be provided with data about a house. The data was collected in 2015 from across the United States. Please answer the question based on the information provided by selecting from one of the two choices. The data provided is enough to reach an answer.
    
The house data is: \n{description}

The question is: What is the price of this house?
The choices are: ['Below $1,500,000', 'Above $1,500,000']

Only respond with JSON output. Do not include any additional words in your answer. Format your response as a JSON file with the keys as provided below:
{{"answer": ""}}
"""
)

house_prices_followup = ("""Now revise the original house data such that your answer to the question on the revised house data would have been {complement}.

Here are the valid options and data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Share the revised house data.

Only respond with JSON output. Do not include any additional words in your answer. Format your response as a JSON file with the keys as provided below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_minimal = ("""Now make a minimal revision to the original house data such that your answer to the question on the revised house data would have been {complement}. 

Your revision should be minimal in the sense it makes the smallest change necessary to change your decision, i.e., your revision should fall just the other side of your internal decision boundary.

Here are the valid options and data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality is computed with Gower’s Distance across all variables. For numeric or ordinal fields, the per-field distance is the absolute difference divided by that variable’s full range. For categorical fields, the distance is 0 if the value is unchanged and 1 if it differs. The overall Gower score is the average of these per-field distances.

Share the revised house data.

Only respond with JSON output. Do not include any additional words in your answer. Format your response as a JSON file with the keys as provided below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

#### Modified versions of the unconstrained followup

You are a research assistant who is being asked to produce twenty alternative wordings of the house-prices counterfactual-elicitation prompt. The variants must all elicit a single counterfactual example that flips the model’s prediction, while not instructing the model to minimise the size of the change.

What must remain unchanged

- Keep the placeholder {complement} exactly as written, (keep the curly-brace syntax).
- Preserve the full list of admissible values and ranges exactly as given for every feature (area values and the integer ranges for bedrooms, bathrooms, floors).
- The four feature names and their spelling: area, bedrooms, bathrooms, floors.
- The instruction that output must be pure JSON with exactly those four keys. Always give {{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}.
- Retain the directive “Do not include any additional words in your answer.” (you should vary the wording slightly, but the prohibition itself must stay).
- Do not mention “smallest”, “minimal”, distances, cost, plausibility, or desirability.
- The order of instructions.
- It must still make it clear that it is "your answer" to the question, i.e. it is the model's predicted label rather than some general predicted label.

What you may rephrase or rearrange

- The opening sentence (e.g. “Now revise the original house data …”) – feel free to swap in synonyms such as modify, adjust, change, alter, or to restructure the sentence.
- The stylistic tone (formal versus plain, active versus passive). It should never be informal.
- The formatting of the feature list – bullets or numbered lines are both okay, provided the values themselves are untouched. You may change the order of the feature list.

What you must not introduce

- Any instruction about minimising changes, measuring distance, or editing the data 'until' the decision flips.
- Additional features, keys, or example values.
- Output formats other than JSON.

Deliverables

- Return twenty distinct prompt texts, each differing in wording while satisfying every rule above. Call these house_prices_followup_0, house_prices_followup_1,...etc
- Return it all as code that can be copy and pasted into Python.

Original prompt

house_prices_followup = ("""Now revise the original house data such that your answer to the question on the revised house data would have been {complement}.

Here are the valid options and data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Share the revised house data.

Only respond with JSON output. Do not include any additional words in your answer. Format your response as a JSON file with the keys as provided below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")


In [15]:
house_prices_followup_0 = ("""Please adjust the original house listing so that your answer to the question on the modified data would have been {complement}.

Here are the valid options and data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Share the revised house data.

Respond only with JSON output. Do not include any additional words in your answer. Structure your reply as a JSON object with the keys provided below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_1 = ("""Modify the initial house features such that your answer to the updated data would yield {complement}.

The permissible values and their data types are listed below.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Provide the revised house data.

Return your answer solely in JSON form. Do not add any further words. Format it as a JSON document using the keys shown:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_2 = ("""Change the given house details so that, for the altered record, your answer would be {complement}.

Below are the admissible values and their data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Present the revised house data.

Your response must consist exclusively of JSON. Refrain from including any extra words. Format your output with the following keys:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_3 = ("""Re-specify the house parameters so that on the revised inputs your answer would equal {complement}.

Acceptable values and types follow.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Share the adjusted house data.

Please output only JSON. Do not append any additional words. Use the key structure shown here:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_4 = ("""Compose an adjusted set of house attributes which would lead your answer to be {complement}.

Valid categories and types are provided below.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Return the modified house data.

Answer with JSON alone. Add no other words. Structure the JSON with these keys:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_5 = ("""Prepare an alternative collection of house features that would cause your predicted label to be {complement}.

The allowed values with types are as follows.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Deliver the revised house details.

Give your reply in JSON only. Do not include any extra words. Use the key template here:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_6 = ("""Construct a counterexample of the house data for which your answer would instead be {complement}.

Here is the catalogue of permitted values and their types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Supply the adjusted house data.

Respond purely with JSON. Insert no supplementary words. Employ the following keys in the JSON:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_7 = ("""Generate a single modified house record such that your answer would return {complement}.

List of allowable values and data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Provide the updated house data.

Your output should be JSON exclusively. Avoid any additional words. Adhere to this key structure:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_8 = ("""Provide a changed instance of the house data ensuring your answer becomes {complement}.

Admissible values and associated types are below.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Present the changed house data.

Return JSON only, with no other words. Use the key format provided:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_9 = ("""Supply one alternative description of the house that would make your answer read {complement}.

Acceptable values with their types are enumerated below.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Share the altered house data.

Produce your response solely as JSON. Do not add extra words. Include exactly these keys:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_10 = ("""Offer a revised set of house characteristics so that your answer would be {complement}.

Below is the roster of permissible values and corresponding types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Give the revised house data.

Output must be JSON only; introduce no additional words. Use the key layout specified:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_11 = ("""Produce an edited version of the house specification leading your answer to {complement}.

The list of valid values and types appears next.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Return the edited house data.

Respond exclusively in JSON and refrain from adding other words. Keep the following keys:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_12 = ("""Design a different house example that would result in your answer being {complement}.

The following are the authorised values and their types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Provide the altered house data.

Supply JSON output only. No extra words are permitted. Apply the given keys:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_13 = ("""Give an adjusted house profile so that your answer evaluates to {complement}.

Here follows the set of valid values and types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Deliver the modified house data.

Use JSON alone in your answer. Do not include other words. Maintain this key schema:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_14 = ("""Craft a single alternative house entry where your answer would turn into {complement}.

These are the valid options and their data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Return the adjusted house data.

Answer only with JSON. Avoid any additional wording. Format with the keys below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_15 = ("""Present a modified house configuration that would yield {complement} as your answer.

The authorised values and types are indicated next.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Supply the changed house data.

Your answer must be in JSON alone; add no further words. The keys are:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_16 = ("""Deliver a changed house dataset such that your answer would become {complement}.

Presented next are the allowed values and corresponding types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Present the revised house data.

Provide JSON output only and omit any extra words. Use the keys that follow:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_17 = ("""Formulate another house listing for which your answer would be {complement}.

Permissible values and types are listed here.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Share the updated house data.

Respond in JSON format exclusively; refrain from extra wording. Include these keys:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_18 = ("""Create one adjusted house example whose answer would be {complement}.

See below for the valid values and data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Provide the modified house data.

Return only JSON. Do not include additional words. Use the keys supplied:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_followup_19 = ("""Illustrate an alternative house scenario causing your answer to equal {complement}.

Valid options with their types are as follows.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Deliver the revised house data.

Output exclusively JSON, with no extra words. Employ the keys shown here:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")


#### Modified versions of the minimal followup

You are a research assistant who is being asked to produce twenty alternative wordings of the house-prices minimal counterfactual-elicitation prompt. Each variant must still ask the model to return one example that changes the predicted label to {complement} and explicitly instructs the model to make the smallest edit required to flip its decision, in line with the Gower-distance definition of minimality.

What must remain unchanged

- Keep the placeholder {complement} exactly as written, (keep the curly-brace syntax).
- Preserve the full list of admissible values and ranges exactly as given for every feature (area values and the integer ranges for bedrooms, bathrooms, floors).
- The four feature names and their spelling: area, bedrooms, bathrooms, floors.
- The instruction that output must be pure JSON with exactly those four keys. Always give {{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}.
- Retain the directive “Do not include any additional words in your answer.” (you should vary the wording slightly, but the prohibition itself must stay).
- Retain the minimality requirement: the prompt must say the revision is “minimal”, “smallest”, or an equivalent term, and must specify that minimality is judged using Gower’s Distance as described in the original prompt. The definition (numeric/ordinal distance, categorical 0/1, average across fields) must stay intact, though wording can change.
- Keep the explanation that the revision should fall “just the other side of your internal decision boundary” or an equivalent phrase that makes it clear minimality is relative to the model’s own boundary.
- The order of instructions.
- The definition of Gower's distance.
- It must still make it clear that it is "your answer" to the question, i.e. it is the model's predicted label rather than some general predicted label.

What you may rephrase or rearrange

- The opening sentence (e.g. “Now revise the original house data …”) – feel free to swap in synonyms such as modify, adjust, change, alter, or to restructure the sentence.
- The stylistic tone (formal versus plain, active versus passive). It should never be informal.
- You may use synonyms for “smallest” or “minimal”, provided the concept is preserved.
- The formatting of the feature list – bullets or numbered lines are both okay, provided the values themselves are untouched. You may change the order of the feature list.

- The opening sentence that asks for a minimal revision; synonyms such as modify, adjust, change, or alter are acceptable.
The stylistic tone (formal versus plain, active versus passive).

What you must not introduce

- Any instruction about minimising changes, measuring distance, or editing the data 'until' the decision flips.
- Additional features, keys, or example values.
- Output formats other than JSON.
- Any instruction that contradicts minimality (e.g. telling the model not to worry about the size of the change).
- Avoid phrases that imply proitising minimality over validity e.g. "tiniest way". There is a clear objective - make the minimal edit required to flip the decision.

Deliverables

- Return twenty distinct prompt texts, each differing in wording while satisfying every rule above. Call these house_prices_minimal_followup_0, house_prices_minimal_followup_1,...etc
- Return it all as code that can be copy and pasted into Python.

Original prompt

house_prices_followup_minimal = ("""Now make a minimal revision to the original house data such that your answer to the question on the revised house data would have been {complement}. 

Your revision should be minimal in the sense it makes the smallest change necessary to change your decision, i.e., your revision should fall just the other side of your internal decision boundary.

Here are the valid options and data types.

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality is computed with Gower’s Distance across all variables. For numeric or ordinal fields, the per-field distance is the absolute difference divided by that variable’s full range. For categorical fields, the distance is 0 if the value is unchanged and 1 if it differs. The overall Gower score is the average of these per-field distances.

Share the revised house data.

Only respond with JSON output. Do not include any additional words in your answer. Format your response as a JSON file with the keys as provided below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")


In [16]:
house_prices_minimal_followup_0 = ("""Adjust the initial house listing in a minimal manner so that your answer for the revised listing would have been {complement}.

The change must be the smallest possible—just beyond your own decision boundary—so the revision is truly minimal.

Here are the permissible values and data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality is evaluated with Gower’s Distance over all variables. For numeric or ordinal fields, the per-field distance equals the absolute difference divided by that variable’s full range. For categorical fields, the distance is 0 when unchanged and 1 when altered. The overall Gower score is the average of these per-field distances.

Provide the revised house data.

Important: Do not include any additional words in your answer. Return only valid JSON in exactly this structure:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_1 = ("""Modify the original dwelling attributes just enough that your answer would switch to {complement}.

Your alteration must be the least-size adjustment required—falling immediately on the other side of your internal decision boundary.

Here are the valid options and data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality is defined by Gower’s Distance. For numeric or ordinal variables the distance is |Δ| divided by the variable’s range; for categorical ones it is 0 if unchanged, 1 if different. The mean of these distances gives the overall score.

Share the altered record now.

Do not include any additional words in your answer. Respond solely with JSON formatted exactly like this:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_2 = ("""Change the given house data with the smallest feasible edit such that your answer on the modified data would become {complement}.

The revision must sit just beyond your own boundary of decision, achieving minimal Gower-distance change.

Here are the admissible values and their data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower’s Distance determines minimality: for numeric/ordinal fields use absolute difference divided by range; for categorical fields distance is 0 if unchanged, 1 if altered; average these distances for the overall score.

Provide the revised entry.

Do not include any additional words in your answer. Your output must be pure JSON in this exact form:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_3 = ("""Please revise the original house record so minimally that the outcome would instead be {complement}.

Ensure the adjustment crosses your decision boundary by the narrowest margin possible.

Here are the acceptable values and data types for every feature:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality is judged with Gower’s Distance: (i) numeric/ordinal distance = |Δ| / full range, (ii) categorical distance = 0 if same, 1 if different; the final score is their mean.

Return the modified record.

Do not include any additional words in your answer. Output only JSON with the keys shown:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_4 = ("""Alter the initial property attributes in the slightest way needed so that your resulting answer would read {complement}.

This change must be minimal—exactly over your internal decision threshold.

The valid domains and types are:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower’s Distance sets minimality: for numeric or ordinal variables, divide the absolute change by the variable’s span; for categorical, use 0 (unchanged) or 1 (changed); average all four distances.

Provide the revised house information.

Do not include any additional words in your answer. Your response must be valid JSON exactly like:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_5 = ("""Kindly adjust the prior house specification just enough so that your answer would flip to {complement}.

The alteration should be minimal in the Gower-distance sense—barely crossing your decision boundary.

Below are the permissible values and their data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality: Gower’s Distance computes per-field distances as |Δ|/range for numeric/ordinal, and 0 or 1 for categorical; the mean of these four gives the overall value.

Submit the modified data.

Do not include any additional words in your answer. Return only this JSON schema:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_6 = ("""Change the original listing by the least possible amount so that your predicted label would be {complement}.

Your update must sit just across your own classification frontier, satisfying minimality.

Permitted values and data types are as follows:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

The smallest change is defined using Gower’s Distance: absolute difference divided by full range for numeric/ordinal fields, and 0/1 for categorical; average across all four variables.

Deliver the updated record.

Do not include any additional words in your answer. Provide pure JSON in precisely this shape:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_7 = ("""Please tweak the original house features minimally so that your answer for the new instance would become {complement}.

Ensure the adjustment is the smallest under Gower’s Distance—lying just the other side of your decision threshold.

Acceptable domains and data types are listed below:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality calculation: numeric/ordinal distance = |Δ| ÷ range; categorical distance = 0 (same) or 1 (different); average for total Gower score.

Provide the modified entry.

Do not include any additional words in your answer. Output only JSON in the following structure:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_8 = ("""Refine the given property attributes by the minimal amount required such that your answer would instead be {complement}.

This revision must straddle your internal boundary with the lowest possible Gower-distance shift.

Here are the valid entries and their data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower’s Distance measure: for numeric/ordinal fields use |Δ| / range; for categorical fields use 0 or 1 as appropriate; the mean gives the overall distance.

Submit the new record.

Do not include any additional words in your answer. Return only JSON formatted exactly like:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_9 = ("""Kindly execute a minimal amendment to the house data so that your classification flips to {complement}.

Keep the edit as small as allowable—just beyond your decision boundary per Gower-distance criteria.

Permissible values and types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower’s Distance rules: numeric/ordinal distance = |Δ| / full range; categorical distance = 0 unchanged, 1 differed; overall distance = average of the four.

Return the altered house data.

Do not include any additional words in your answer. Output pure JSON in this format only:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_10 = ("""Alter the source house attributes with the least modification required for your answer to become {complement}.

Your change must land just over your internal decision boundary, thus meeting the minimal Gower-distance criterion.

Valid ranges and types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower metric: (numeric/ordinal) distance = abs(change)/range; (categorical) distance = 0 if same, 1 if changed; mean of the four = overall distance.

Present the revised listing.

Do not include any additional words in your answer. Respond solely with JSON shaped exactly like:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_11 = ("""Please modify the previous house entry in the smallest way that would cause your predicted label to be {complement}.

The revision should sit just across your decision boundary according to Gower’s Distance minimality.

Below, observe all valid values and their types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality details: numeric/ordinal distance = |Δ| divided by range; categorical distance = 0 (no change) or 1 (change); average them for total Gower distance.

Return the altered data now.

Do not include any additional words in your answer. Your output must be pure JSON matching:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_12 = ("""Make a minimal adjustment to the house record so that your decision would have been {complement}.

Ensure the shift is the least possible just past your classification boundary by Gower’s Distance.

Acceptable values and data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Definition of minimality: numeric/ordinal → |Δ|/range; categorical → 0 if unchanged, 1 if different; overall distance = arithmetic mean.

Provide the updated house details.

Do not include any additional words in your answer. Return JSON exactly like the following:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_13 = ("""Update the original house data with the minimal possible change such that your resulting answer becomes {complement}.

The edit should be just sufficient to pass your own decision boundary, maintaining Gower-minimality.

Permitted values and types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Use Gower’s Distance to gauge minimality: per-field numeric/ordinal = |Δ| / range; categorical = 0 or 1; mean of all equals overall.

Submit the changed record.

Do not include any additional words in your answer. Output only the JSON object shown here:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_14 = ("""Kindly perform the least alteration to the provided house attributes so that your predicted label flips to {complement}.

This adjustment must lie immediately beyond your decision boundary, minimising Gower-distance.

Valid values & data types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality formula: numeric or ordinal distance = abs(change)/range; categorical distance = 0 when unchanged, 1 when changed; average gives overall Gower score.

Return the tweaked data.

Do not include any additional words in your answer. Provide JSON only, using the keys below:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_15 = ("""Revise the initial property listing by the smallest increment necessary so that your verdict would be {complement}.

Your change should just traverse your internal decision boundary, preserving minimality via Gower’s Distance.

Allowable values and their types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower’s Distance: numeric/ordinal = |Δ| ÷ full range; categorical = 0 if same, 1 if different; average of four = total distance.

Supply the modified record.

Do not include any additional words in your answer. Respond with JSON, exactly:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_16 = ("""Please craft a minimal change to the house data such that your classification outcome would have been {complement}.

The edit must be the smallest permitted, crossing your decision boundary only marginally according to Gower’s Distance.

See the allowed values and data types below:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Minimality computation: (numbers/ordinals) |Δ| / range; (categoricals) 0 or 1; the average yields overall Gower distance.

Return the adjusted record now.

Do not include any additional words in your answer. Output only JSON in this exact structure:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_17 = ("""Execute the least necessary alteration to the given house features so that your answer would read {complement}.

Ensure the modification passes your decision threshold by the minimum Gower-distance increment.

Permitted values (types specified):

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1‒5 inclusive (integer)
bathrooms: 1‒4 inclusive (integer)
floors: 1‒4 inclusive (integer)

Gower’s Distance details: numeric/ordinal per-field distance = |Δ| / range; categorical per-field distance = 0 unchanged, 1 changed; overall distance = mean.

Present the revised data.

Do not include any additional words in your answer. Return JSON formatted exactly as:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_18 = ("""Alter the house specification with the minimum edit needed such that your result becomes {complement}.

This adjustment should rest just beyond your internal boundary, minimising Gower-distance.

Below are the admissible entries and types:

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1‒5 inclusive (integer)
bathrooms: 1‒4 inclusive (integer)
floors: 1‒4 inclusive (integer)

Minimality metric (Gower): numeric/ordinal distance = abs difference ÷ range; categorical distance = 0 if value unchanged, 1 otherwise; mean of distances gives total.

Provide the altered listing.

Do not include any additional words in your answer. Respond only with JSON in exactly this form:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")

house_prices_minimal_followup_19 = ("""Make the most minor possible adjustment to the original house data so that your predicted label flips to {complement}.

The change must merely tip over your own decision boundary while remaining Gower-minimal.

Valid values (with data types):

area: ['500', '1000', '1500', '2000', '2500', '3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500', '9000', '9500', '10000'] (ordinal)
bedrooms: 1-5 inclusive (integer)
bathrooms: 1-4 inclusive (integer)
floors: 1-4 inclusive (integer)

Gower’s Distance governs minimality: per-field numeric/ordinal = |Δ| / full range; per-field categorical = 0 (same) or 1 (different); average all four for the overall score.

Submit the modified data now.

Do not include any additional words in your answer. Provide JSON exactly structured as:
{{"area": "", "bedrooms": "", "bathrooms": "", "floors": ""}}
""")


#### Adding these datasets

In [17]:
# inital dictionary
variations_dict = {}

# Make lots of new templates
for i in range(20):
    d = {f"house_prices_{i}": {
        "local":True,
        "filepath":"data/house_prices", 
        "name":"",
        "split":"",
        "options":['Below $1,500,000',
        'Above $1,500,000'],
        "answer_key": "answer",
        "input_key":"description",
        "standard_prompt_template":house_prices,
        "followup_template":globals()[f"house_prices_followup_{i}"],
        }}
    variations_dict.update(d)

# Make lots of new templates
for i in range(20):
    d = {f"house_prices_minimal_{i}": {
        "local":True,
        "filepath":"data/house_prices", 
        "name":"",
        "split":"",
        "options":['Below $1,500,000',
        'Above $1,500,000'],
        "answer_key": "answer",
        "input_key":"description",
        "standard_prompt_template":house_prices,
        "followup_template":globals()[f"house_prices_minimal_followup_{i}"],
        }}
    variations_dict.update(d)

# updates the datasets
datasets.update(variations_dict)

In [18]:
# save
with open(REPO_ROOT / 'src/models_datasets/datasets.json', 'w') as json_file:
    json.dump(datasets, json_file, indent=4)